In [ ]:
import tifffile
import pandas
import os 
import numpy as np
from collections import Counter
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns

from griottes.analyse import cell_property_extraction
from griottes.graphmaker import graph_generation_func
from griottes.graphplotter import graph_plot
import griottes

import urllib3
import shutil

# Load image and mask of nuclei

From the image we can extract the single cell information. For the analysis, it is necessary that the channels be structured as $C \times Z \times Y \times X$, where $C$ is the fluorescence channels in a multi-channel image.

The file `3D_spheroid_multichannel_image.tiff` can be downloaded from by executing the box below.

In [ ]:

url = 'https://github.com/BaroudLab/Griottes/releases/download/v1.0-alpha/3D_spheroid_multichannel_image.tif'
filename = '3D_spheroid_multichannel_image.tif'

if not os.path.exists(filename):
    c = urllib3.PoolManager()

    with c.request('GET',url, preload_content=False) as resp, open(filename, 'wb') as out_file:
        shutil.copyfileobj(resp, out_file)

    resp.release_conn()
else:
    print('dataset already exists')

In [ ]:
spheroid_image = tifffile.imread('3D_spheroid_multichannel_image.tif')

# Extract single-cell information from the image

One can extract properties from the cells by running the `cell_property_extraction.get_cell_properties` module.

## Extracting single-cell fluorescence data
 
This is done by setting `analyze_fluo_channels` to `True`. Then multiple methods exist to measure the fluorescence proper. They are tuned by modifying `fluo_channel_analysis_method` option to `basic`, `local_voronoi` or `local_sphere`.

 - `basic` measures the fluorescence inside the cell mask. This method relies on the `skimage.regionprops` and is the quickest of the three.

 - `local_voronoi` generates a Voronoi tesselation of the tissue and measures the fluorescence inside the cell voxel.

 - `local_sphere` measures the fluorescence within a sphere of radius `radius` around the cell center.

In [ ]:
prop = cell_property_extraction.get_cell_properties(
    spheroid_image,
    mask_channel = 3,
    analyze_fluo_channels = True,
    fluo_channel_analysis_method = 'basic',
    percentile = 10
    )

## Extracting geometric data
 
This is done by setting `cell_geometry_properties` to `True`. 

In [ ]:
prop = cell_property_extraction.get_cell_properties(
    spheroid_image,
    mask_channel = 3,
    analyze_fluo_channels = False,
    cell_geometry_properties = True,
    )

This module possesses multiple other options accessible by calling:

In [ ]:
help(cell_property_extraction.get_cell_properties)

# Network representation of tissues in 3D

From the extracted positions of the cell centers, it is possible to generate a network representation of the 3D spheroid. We have extracted the positions and properties of multiple spheroids that are available by executing the cells immediately below. These MSC spheroids have been incubated for 29 hours before being imaged and we detect cell type by comparing the intensity of the staining.

For illustration purposes we decide to attribute different cell types depending on the fluorescence measured within the mask. Any cell which `mean_intensity_1` is above the median is considered CD146+, otherwise it is CD146-.

In [ ]:
url = 'https://github.com/BaroudLab/Griottes/releases/download/v1.0-alpha/single_spheroid_data.csv'
filename = 'single_spheroid_data.csv'

if not os.path.exists(filename):
    c = urllib3.PoolManager()

    with c.request('GET',url, preload_content=False) as resp, open(filename, 'wb') as out_file:
        shutil.copyfileobj(resp, out_file)

    resp.release_conn()
else:
    print('dataset already exists')

In [ ]:
single_spheroid_data = pandas.read_csv('single_spheroid_data.csv')

In [ ]:
single_spheroid_data['cell_type'] = (single_spheroid_data.mean_intensity_1 > single_spheroid_data.mean_intensity_1.median()).astype(int)
single_spheroid_data.index = np.arange(len(single_spheroid_data))

legend_list = ['CD146-', 'CD146+']
single_spheroid_data['legend'] = [legend_list[single_spheroid_data.loc[i, 'cell_type']] for i in range(len(single_spheroid_data))]

color_list = [plt.cm.Set3(i) for i in range(2)]
single_spheroid_data['color'] = [color_list[single_spheroid_data.loc[i, 'cell_type']] for i in range(len(single_spheroid_data))]


## Delaunay-based network construction

To represent the spheroid from point-data we use the Delaunay-rule to build the spheroid.

In [ ]:
descriptors = ['x', 'y', 'legend', 'color', 'cell_type']

G = graph_generation_func.generate_delaunay_graph(single_spheroid_data, 
                                                 descriptors = descriptors, 
                                                 distance = 100,
                                                 image_is_2D = False)

From the graph G it is possible to plot a visual representation of the network. Griottes contains several specific plotting functions adapted for the network representation of tisssues. These functions are called through the graph_plot module. Here we call a specific function for 3D plots.

In [ ]:
graph_plot.network_plot_3D(G, 
                figsize = (7, 7),
                alpha_line=0.6,
                scatterpoint_size=10,
                legend=True,
                legend_fontsize = 18,
                theta = -0,
                psi = -0,
                xlim = (prop.x.min() - 5, prop.x.max() + 5),
                ylim = (prop.y.min() - 5, prop.y.max() + 5),
                zlim = (prop.z.min() - 5, prop.z.max() + 5))

## Attributing layers to spheroids

Each cell can be attributed a layer corresponding to the number of links connects each cell to a cell on the outside of the spheroid.

In [ ]:
G = graph_generation_func.attribute_layer(G)

layer_dict = nx.get_node_attributes(G, 'layer')

for ind in single_spheroid_data.index:
    single_spheroid_data.loc[ind, 'layer'] = int(layer_dict[ind])

for ind in single_spheroid_data.index:
    
    if single_spheroid_data.loc[ind, 'layer'] < 2:
        
        single_spheroid_data.loc[ind, 'color'] = 'r'
        single_spheroid_data.loc[ind, 'legend'] = 'Outer layer'
        
    else:
        
        single_spheroid_data.loc[ind, 'color'] = 'b'
        single_spheroid_data.loc[ind, 'legend'] = 'Inner layers'
        
descriptors = ['x', 'y', 'legend', 'color', 'cell_type']

G = graph_generation_func.generate_delaunay_graph(single_spheroid_data, 
                                                 descriptors = descriptors, 
                                                 distance = 100)

In [ ]:
graph_plot.network_plot_3D(G, 
                figsize = (7, 7),
                alpha_line=0.6,
                scatterpoint_size=10,
                legend=True,
                legend_fontsize = 18,
                theta = -0,
                psi = -0,
                xlim = (prop.x.min() - 5, prop.x.max() + 5),
                ylim = (prop.y.min() - 5, prop.y.max() + 5),
                zlim = (prop.z.min() - 5, prop.z.max() + 5))

# Comparing graph properties between spheroids

We download single-cell data from multiple MSC spheroids that have incubated for 29 hours. This will allow us to look at spheroid to spheroid variability within the experiment.

In [ ]:
url = 'https://github.com/BaroudLab/Griottes/releases/download/v1.0-alpha/multiple_spheroid_data_t29.csv'
filename = 'multiple_spheroid_data_t29.csv'

if not os.path.exists(filename):
    c = urllib3.PoolManager()

    with c.request('GET',url, preload_content=False) as resp, open(filename, 'wb') as out_file:
        shutil.copyfileobj(resp, out_file)

    resp.release_conn()
else:
    print('dataset already exists')
    
multiple_spheroid_data = pandas.read_csv('multiple_spheroid_data_t29.csv')

In [ ]:
def make_graphs(dataframe,
                descriptors,
                distance = 70):
    
    graph_list = []
    
    for spheroid_number in dataframe.spheroid_number.unique():
        
        loc_frame = dataframe[dataframe.spheroid_number == spheroid_number]
        
        G = graph_generation_func.generate_delaunay_graph(single_spheroid_data, 
                                                 descriptors = descriptors, 
                                                 distance = distance)

        G = graph_generation_func.attribute_layer(G)

        layer_dict = nx.get_node_attributes(G, 'layer')
        
        graph_list.append(G)
    
    return graph_list

def get_neighbour_properties(G,
                             neighbour_descriptor):
    
    """
    
    This function iterates through the cells composing the spheroid
    and records the distribution of features with regard to each other
    in the tissue.
    
    """

    resultframe = pandas.DataFrame()
    i = 0
        
    neighbour_descriptors = nx.get_node_attributes(G, neighbour_descriptor)

    for node in G.nodes:
        
        resultframe.loc[i, 'label'] = node
        resultframe.loc[i, neighbour_descriptor] = neighbour_descriptors[node]
        resultframe.loc[i, 'degree'] = len([n for n in G.neighbors(node)])
        
        neighbour_props = [neighbour_descriptors[n] for n in G.neighbors(node)]
        neighbour_props_counter = Counter(neighbour_props)
                
        for key in neighbour_props_counter.keys():
            resultframe.loc[i, key] = neighbour_props_counter[key]
            
        i += 1

    return resultframe.fillna(0)


Here we choose to compare the degree distribution between spheroids.

In [ ]:
descriptors = ['mean_intensity_1', 'color', 'legend']

legend_list = ['CD146-', 'CD146+']
multiple_spheroid_data['legend'] = [legend_list[multiple_spheroid_data.loc[i, 'color_int']] for i in range(len(multiple_spheroid_data))]

color_list = [plt.cm.Set2(i) for i in range(2)]
multiple_spheroid_data['color'] = [color_list[multiple_spheroid_data.loc[i, 'color_int']] for i in range(len(multiple_spheroid_data))]

graph_list = make_graphs(multiple_spheroid_data,descriptors)

In [ ]:
all_graph_neighbourhood_props = pandas.DataFrame()

for graph in graph_list:
    
    loc_neighbourhood_props = get_neighbour_properties(graph,
                             neighbour_descriptor = 'legend')
    all_graph_neighbourhood_props = all_graph_neighbourhood_props.append(loc_neighbourhood_props)
    


In [ ]:
fig, ax = plt.subplots(figsize = (5,4))

ax.tick_params(axis='both', which='major', labelsize=14)
ax.tick_params(axis='both', which='minor', labelsize=14)

g = sns.histplot(loc_neighbourhood_props, 
             x="degree",
             stat = 'probability',
             palette='Set2',
             kde = False,
             cumulative=False,
             common_norm=False,
             binwidth = 1,
             alpha = 0.6)

sns.kdeplot(data=all_graph_neighbourhood_props, 
            x="degree",
            c = 'r',
            ls = '--',
            lw = 4)

ax.set_ylabel('probability', fontsize = 15)
ax.set_xlabel('degree', fontsize = 15)
plt.tight_layout()